In [1]:
versioninfo()

Julia Version 1.4.1
Commit 381693d3df* (2020-04-14 17:20 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-4770 CPU @ 3.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, haswell)
Environment:
  JULIA_HOME = /home/tate/.local/share/julia-1.4.1/bin


In [2]:
using LinearAlgebra
using DelimitedFiles

In [3]:
# d0(λ)の値を計算

λ = 10.0 .^ LinRange(-16, 16, 201)
μ = @. log(λ)^2 + 2*π^2
d = @. asin(sqrt((μ - sqrt(μ^2 - 4*π^4)) / (2*π^2)))

Data = [λ d]
writedlm("data_d0lambda.txt", Data)

In [25]:
# ϕ(κ)の値を計算

function get_lr(κ; ϵ=2.0^(-53))
    # Step 2-3 in [Alg. 1, Tatsuoka et al., 2020]
    λmax = sqrt(κ)
    λmin = 1/λmax
    norm_A_inv = λmax
    norm_A_minus_I = max(abs(λmax-1), abs(λmin-1))
    θ = log(λmax)
    
    # Step 4-7
    ϵmax = 3 / θ * (norm_A_minus_I * norm_A_inv) / (1 + norm_A_inv)
    if ϵ > ϵmax
        ϵ = ϵmax / 2
    end
    
    # Step 8 and 10
    a1 = θ * ϵ / 3 / norm_A_minus_I
    a2 = 1 / 2 / norm_A_minus_I
    if a1 < a2
        atanh_2a_minus_1 = (log(2a1) - log(2) - log1p(-a1)) / 2
        l = asinh(2/π * atanh_2a_minus_1)
    else
        l = asinh(2/π * atanh(2*a2-1))
    end
    
    # Step 9 and 11
    δ = θ * ϵ / 3 / norm_A_minus_I / norm_A_inv
    b1 = 1 - δ
    b2 = 2 * norm_A_inv / (2*norm_A_inv+1)
    if b1 > b2
        atanh_2b_minus_1 = (log(2) + log1p(2δ) - log(2δ)) / 2
        r = asinh(2/π * atanh_2b_minus_1)
    else
        r = asinh(atanh(2*b2-1))
    end
    
    return l, r
end


function speed_de(κ)
    λmax = sqrt(κ)
    μ = log(λmax)^2 + 2π^2
    d = asin(sqrt((μ - sqrt(μ^2 - 4*π^4)) / (2*π^2)))
    l, r = get_lr(κ)
    return 2 * π * d / (r-l)
end


function speed_gl(κ)
    λmax = sqrt(κ)
    τ = (sqrt(λmax)+1) / abs(sqrt(λmax)-1)
    return 2 * log(τ)
end


κ = 10.0 .^ LinRange(0, 16, 101)[2:end]
gl = [speed_gl(κ_i) for κ_i in κ]
de = [speed_de(κ_i) for κ_i in κ]
Data = [κ gl de]
writedlm("data_convspeed.txt", Data)

In [26]:
# 2分法でGLとDEの速度が同じになる点を求める

function find_intersection(left, right)
    center = 1.0
    for _ = 1:10^6
        center = (left + right)/2
        f = speed_gl(center) - speed_de(center)
        if f < 0
            right = center
        else
            left = center
        end
    end
    return center
end

intersect_x = find_intersection(1e+2, 1e+5)
@show intersect_x
@show speed_de(intersect_x)
@show speed_gl(intersect_x);

intersect_x = 2711.919364407967
speed_de(intersect_x) = 0.5578842198050705
speed_gl(intersect_x) = 0.5578842198050702
